In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
diabete = pd.read_csv("/kaggle/input/pyms-diabete/diabete.csv")

In [ ]:
diabete.head().T

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "glucose", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "insulin", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "tension", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "thickness", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(diabete, hue="diabete", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "bmi", shade=True)
fig.add_legend()

In [ ]:
sns.jointplot("bmi","diabete",diabete,kind='kde')

Quelle colonnes sont à garder ?

In [ ]:
diabete.columns

In [ ]:
diabete.count()

Tout est rempli


### Déséquilibre des distributions

In [ ]:
sns.distplot(diabete.n_pregnant, color='blue')

In [ ]:
sns.distplot(diabete.glucose, color='blue')

In [ ]:
sns.distplot(diabete.tension, color='blue')

In [ ]:
sns.distplot(diabete.thickness, color='blue')

In [ ]:
sns.distplot(diabete.insulin, color='blue')

In [ ]:
sns.distplot(diabete.bmi, color='blue')

In [ ]:
sns.distplot(diabete.pedigree, color='blue')

In [ ]:
sns.distplot(diabete.age, color='blue')

Il est étrange de retrouver des valeurs à 0 pour : BMI, Insulin, thickness, glucose

Je vais passer les valeurs valant 0 à NaN pour mieux visualiser les vides

In [ ]:
df_diabete = diabete.replace(0,np.nan)
df_diabete.n_pregnant = diabete.n_pregnant
df_diabete.pedigree = diabete.pedigree
df_diabete.diabete = diabete.diabete

In [ ]:
df_diabete.count()

Nous allons remplir les valeurs NaN à l'aide de la méthode pad

In [ ]:
df_diabete = df_diabete.fillna(method='pad')

In [ ]:
df_diabete.count()

A priori les trois premiers cas était en NaN ce qui explique qu'il n'ai pas récupérer de valeurs, on s'adapte :

In [ ]:
df_diabete = df_diabete.fillna(value = {'insulin':df_diabete.insulin.mean()})

In [ ]:
df_diabete.count()

In [ ]:
plt.hist(df_diabete.insulin, bins=80)

### Mise à l'échelle des données quantitatives

In [ ]:
df_diabete['log_pedigree'] = np.log(df_diabete.pedigree+1)

In [ ]:
sns.kdeplot(df_diabete.log_pedigree, color='blue')

In [ ]:
df_diabete = df_diabete.drop(['pedigree'], axis=1)

In [ ]:
df_diabete.head()

### Préparation des jeux de test

In [ ]:
df_diabete.diabete.value_counts() 

In [ ]:
X = df_diabete.drop(['diabete'], axis=1)
y = df_diabete.diabete

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1,shuffle=True)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(X_train)
print(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

### Performances

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
print(dfprobas)

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Ajustement des hyperparamètres (Random Forests)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

### Importance des caractéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))